In [ ]:
from langchain_core.prompts import ChatPromptTemplate
# ChatPromptTemplate 
# 대화형 프롬프트를 생성하는 데 사용
# 시스템 메시지와 사용자 메시지를 포함한 구조화된 프롬프트를 정의할 수 있다.

from langchain_core.runnables import RunnablePassthrough
# RunnablePassthrough
# 입력 데이터를 수정하지 않고 그대로 전달하는 "중간 연결점" 역할

In [16]:
# OpenAI
# 환경 변수에서 API_KEY 값 읽기

from dotenv import load_dotenv 
import os 

load_dotenv() 

# .env 파일을 로드 

api_key = os.getenv("API_KEY")


In [17]:
import os
from getpass import getpass


# 환경 변수에서 API_KEY 값 읽기
os.environ["OPENAI_API_KEY"] = api_key

### 모델 로드 하기 

OpenAI 모델

In [18]:

#API 키 오류로 동작안됨

from langchain_openai import ChatOpenAI

# LangChain에서 제공하는 OpenAI의 ChatGPT 모델과 상호작용하기 위한 인터페이스를 제공
# OpenAI API를 기반으로 동작하며, 대화를 생성하거나 특정 명령어를 실행하는 데 사용


from langchain_core.messages import HumanMessage

# LangChain에서 사용하는 메시지 객체 중 하나
# HumanMessage: 사람이 챗모델에 보낸 메시지



# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")


In [19]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("file/인공지능산업최신동향_2024년11월호.pdf")

# 페이지 별 문서 로드
docs = loader.load()

**RecursiveCharacterTextSplitter  
자료의 특성상 문맥 유지가 중요하다고 판단했게때문에   
RecursiveCharacterTextSplitter 청킹방식을 선택**

### RecursiveCharacterTextSplitter

RecursiveCharacterTextSplitter 청킹방식의 경우  

텍스트를 "문단 → 문장 → 단어" 순서로 나누기때문에 문맥을 유지할 수 있으며,    
복잡한 자료에 적합한 청킹방식이다.

단점으로,  
처리속도가 느릴 수 있고, 대규모 데이터에서는 성능 문제가 생길 수 있습니다.

문맥과 의미를 유지해야 하는 고급 챗봇

In [20]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

### CharacterTextSplitter
CharacterTextSplitter는 단순히 일정한 문자 수로 나누는 방법이다.

예를 들어, "한 페이지를 100자씩 나누되 10자씩 겹치게 한다" 고 설정 하면,  
한 청크는 100자로 제한하고, 다음 청크는 앞의 10자를 겹치며 시작하게 한다.

문장을 기준으로 하진 않지만, 필요한 경우 구분자(예: 줄바꿈 \n\n)를 사용해 한 문장이나 문단 단위로 나눌 수 있다.

문자 수로 나누는 방법이기 때문에 문맥 연결이 다소 떨어질 수 있다.
로 나눌 수 있다.

In [ ]:
'''
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(docs)
'''

### 임베딩(Embedding)

자연어 처리(NLP)에서 텍스트 데이터를 수치화하여 머신러닝 모델이 이해할 수 있도록 하기 위한 과정

In [22]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 임베딩 모델을 사용하여 텍스트를 벡터(숫자 배열)로 변환하는 역할
# 주로 텍스트 검색, 추천 시스템, 또는 문서 유사도 계산 등에서 활용


# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# OpenAI의 최신 임베딩 모델인 'text-embedding-ada-002'를 사용하도록 OpenAIEmbeddings 객체를 초기화

# 'text-embedding-ada-002'는 
# 입력된 텍스트를 고차원의 벡터로 변환
# 이 벡터는 텍스트 간의 '의미적 유사성'을 계산하는 데 사용할 수 있다.

### 벡터 스토어(Vector Store)

FAISS를 사용하여 텍스트 데이터를 벡터로 변환하고,  
해당 벡터를 기반으로 특정 쿼리에 대해 빠르게 검색할 수 있는 '벡터 스토어(Vector Store)'를 생성

문서를 벡터로 변환하고, FAISS를 활용하여 유사한 문서를 빠르게 검색할 수 있도록 준비하는 코드

In [23]:
import faiss
# Facebook에서 개발한 FAISS(Facebook AI Similarity Search) 라이브러리
# 대규모 벡터 데이터에서 빠르게 유사한 항목을 검색할 수 있는 라이브러리

from langchain_community.vectorstores import FAISS
# LangChain 라이브러리에서 FAISS를 활용하기 위한 래퍼(wrapper)
# 문서 검색이나 대화형 AI 응용 프로그램을 쉽게 구현할 수 있도록 다양한 벡터 저장소 및 임베딩 모델과의 통합을 제공


vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)
# 문서 데이터를 벡터로 변환하고 이를 FAISS에 저장하여 빠른 검색이 가능하도록 설정
# documents=splits - 텍스트를 청킹하여 나눈 작은 문서들의 리스트
# embedding=embeddings - 임베딩 모델을 사용해 각 문서를 벡터로 변환



### FAISS를 Retriever로 변환

FAISS 벡터 스토어를 검색 리트리버(Retriever)로 변환하여 유사한 문서를 검색할 수 있도록 설정하는 코드


In [24]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

# vector_store.as_retriever() - vector_store 객체를 리트리버로 변환
# search_type="similarity" - 코사인 유사도 또는 유사 벡터 검색을 사용해 쿼리와 문서 벡터 간의 유사도를 계산
#                          - 유사도가 높은 순서대로 검색 결과를 반환
#search_kwargs={"k": 1} - 가장 유사한 문서 1개를 반환하도록 설정
#                       - "k"는 반환할 문서의 개수를 나타냄 ex) k = 5 는 5개를 반환


### 프롬프트 템플릿 정의 
시스템 메시지와 사용자 메시지를 설정하여,   
특정한 질문에 컨텍스트를 기반으로 답변을 생성할 수 있도록 구성됨


In [30]:
# 프롬프트 파일 읽기
def load_prompt_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().strip()

# 프롬프트 파일을 system 메시지로 사용
prompt_file_path = "prompts/prompt_01.txt"
system_prompt_text = load_prompt_from_file(prompt_file_path)

# ChatPromptTemplate에서 system 메시지와 user 메시지를 설정
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_text),  # AI의 동작 방식에 대한 지시
    ("user", "{question}")  # 사용자가 던지는 질문
])

# contextual_prompt = ChatPromptTemplate.from_messages([...])
# ChatPromptTemplate를 통해 프롬프트 템플릿을 정의하고, 이를 contextual_prompt 변수에 저장
# .from_messages([...]) 메서드를 사용하여 메시지를 구성

>system  
       >- 시스템 메시지는 AI의 행동 방식을 지시하는 역할  
       >- "Answer the question using only the following context."  
       >- 명령: '아래 제공된 컨텍스트를 사용하여 질문에만 답하여라'

>User 
     >- 사용자 메시지는 질문 또는 요청의 내용을 나타냄.  
     >- "Context: {context}\\n\\nQuestion: {question}"  
     >- 플레이스 홀더를 사용하여 {context}와 {question}를 실제내용으로 치환

>플레이스 홀더  
     >- 플레이스홀더(Placeholder)는 빈칸이나 변수를 지정하는 텍스트 조각  
    >- 실행 시점에 값을 동적으로 채워 넣기 위해 미리 정의된 자리표시자 역할  
    >- ex) "Hello, {name}! How are you?" 
    >- 위의 경우는 {name}이 플레이스 홀더다.

### RAG 체인 구성

**RAG(검색 기반 생성 모델)** 는 인공지능이 더 정확하고 유용한 답변을 만들 수 있도록 도와주는 방법이다.  
일반적으로 AI는 인터넷 등에서 다른 데이터를 검색하지 않기때문에 최신정보나 구체적인 데이터를 알 수 없다.  

그렇기에 그점을 보완하기 위해 RAG를 사용하는데,  
AI가 답변을 만들기전에 관련된 정보를 인터넷이나 로컬 저장소에 저장된 다른 문서를 참조하여 답변할 수 있게 도와준다.  
즉, AI가 질문에 답하기 전에 "먼저 검색하고, 그 정보를 바탕으로 대답하라"는 방식이다.

In [31]:
from langchain.chains import LLMChain


# 입력 데이터를 그대로 전달하는 간단한 클래스
class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs
# 기본적으로 입력값을 그대로 반환하는 역할
# invoke 메서드는 inputs를 그대로 반환하여 파이프라인에서 다른 처리를 거치지 않고 값을 전달하는 단순한 흐름을 구현



# 문서를 AI가 이해할 수 있는 질문으로 바꿔주는 클래스
class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template  # 질문 템플릿 설정
    
    def invoke(self, inputs):
        # 문서가 여러 개면 하나로 합침
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs  # 문서가 하나면 그대로 사용
        
        # 템플릿을 사용하여 컨텍스트와 질문을 포함한 프롬프트 생성        
        # 프롬프트 템플릿에 적용
        # 템플릿에 따라 질문 생성
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,                # 검색된 문서 컨텍스트
            question=inputs.get("question", "")  # 질문을 추가
        )
        return formatted_prompt                  # 완성된 프롬프트 반환
                                                 # AI가 이해할 수 있는 질문 반환


# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
# Retriever(검색기)를 래핑하여 검색 결과를 제공하는 클래스
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever              # 검색기 설정

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")  # 질문 가져오기
        else:
            query = inputs                      # 그냥 질문 텍스트 받기
        # 검색 수행
        # 검색해서 관련 문서 가져오기
        response_docs = self.retriever.get_relevant_documents(query)
        return response_docs # 검색된 문서 반환


# AI 모델과 질문 연결
llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),         # 질문에 맞는 문서를 찾음
    "prompt": ContextToPrompt(contextual_prompt),   # 문서를 AI가 이해할 수 있는 질문으로 만듦
    "llm": model                                    # 질문을 모델에 전달해 답변 생성
}


In [ ]:

# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요! (종료를 원하시면 '질문종료'라고 입력해주세요!) ")

    if query == '질문종료' or query == '질문 종료':
        print('답변이 종료되었습니다! 감사합니다!')
        break
    else:
        # 1. Retriever로 관련 문서 검색
        response_docs = rag_chain_debug["context"].invoke({"question": query})
        # 사용자가 입력한 질문(query)을 검색 시스템으로 넘겨서 관련 문서를 찾음
        
        # 2. 문서를 프롬프트로 변환
        # 문서를 AI가 이해하기 쉬운 질문으로 바꾸는 단계
        prompt_messages = rag_chain_debug["prompt"].invoke({
            "context": response_docs,  # 검색한 문서들
            "question": query          # 사용자가 입력한 질문
        })
        
        # 3. LLM으로 응답 생성
        # AI가 최종 답변을 생성하는 단계
        response = rag_chain_debug["llm"].invoke(prompt_messages)
        # 정리된 질문을 AI에게 넘겨서 답변 생성
        
        print("\n답변:")
        print(response.content)

질문을 입력하세요! (종료를 원하시면 '질문종료'라고 입력해주세요!)  안녕



답변:
안녕하세요! 어떤 도움을 드릴까요? 필요한 내용이나 질문이 있다면 말씀해 주세요.


질문을 입력하세요! (종료를 원하시면 '질문종료'라고 입력해주세요!)  최근 인공지능 동향에 대해 알려줘



답변:
최근 인공지능 동향에 대한 정보를 요약하여 제공합니다.

1. **대규모 언어 모델**: OpenAI의 GPT-4와 같은 대규모 언어 모델의 발전이 두드러지며, 자연어 처리(NLP) 분야에서의 활용이 증가하고 있습니다. 이러한 모델은 다양한 언어 작업을 수행할 수 있는 능력을 가지고 있습니다.

2. **AI 윤리 및 규제**: 인공지능의 윤리적 사용과 관련된 논의가 활발해지고 있으며, 정부와 기업이 AI의 안전하고 공정한 사용을 보장하기 위한 규제 방안을 마련하고 있습니다.

3. **AI의 산업 응용**: 인공지능이 의료, 금융, 제조업 등 다양한 산업에 통합되고 있으며, 데이터 분석, 예측 모델링, 자동화된 의사결정 등에서 그 가치를 발휘하고 있습니다.

4. **AI와 창의성**: AI가 예술, 음악, 글쓰기 등 창의적인 분야에서도 활용되면서, 인간의 창작 활동과 AI의 협업에 대한 관심이 증가하고 있습니다.

5. **인공지능의 지속 가능성**: AI 모델의 훈련과 운영에 필요한 에너지 소비 문제에 대한 우려가 커지면서, 지속 가능한 AI 개발을 위한 연구와 기술 혁신이 필요하다는 목소리가 높아지고 있습니다.

6. **AI의 접근성 확대**: 다양한 플랫폼과 도구가 개발되어, 비전문가도 쉽게 AI 기술을 활용할 수 있는 환경이 조성되고 있습니다. 이는 AI의 민주화에 기여하고 있습니다.

이와 같은 동향은 인공지능 기술의 발전과 그에 따른 사회적, 경제적 변화를 반영하고 있습니다.


### 일반 LLM 답변


**소프트웨어 엔지니어링에서의 AI**  
AI는 코드 작성, 디버깅 및 개발 생애주기를 개선하여 소프트웨어 엔지니어링의 효율성을 높이고 있으며, 특히 프롬프트 엔지니어링이 중요한 기술로 자리 잡고 있습니다​

**AI와 윤리**  
AI의 윤리적 문제는 데이터 투명성, 책임감 있는 사용 및 설명 가능한 AI를 포함해 점점 더 중요한 관심사가 되고 있습니다​

**AI와 헬스케어**  
AI는 질병 위험 예측 및 예방적 치료 추천을 통해 헬스케어 분야에서 혁신적인 변화를 일으키고 있습니다​

**AI 생성 콘텐츠 및 예술**  
AI를 이용한 이미지와 예술 창작이 확산되면서 원작자 보호와 무단 사용에 대한 논란이 일어나고 있습니다​

**양자 컴퓨팅과 AI**  
양자 컴퓨팅과 AI의 결합은 분자 시뮬레이션, 최적화 문제 해결 등에서 큰 잠재력을 보여주고 있습니다​
